# Callback example for permutation flow shop

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/PyJobShop/PyJobShop/blob/main/examples/permutation_flow_shop.ipynb)

> If you're using this notebook in Google Colab, be sure to install PyJobShop first by executing ```pip install pyjobshop``` in a cell.

In this notebook, we demonstrate how to use a callback function when solving a permutation flow shop problem (PFSPs) using PyJobShop. The first instance given in the notebook "permutation_flow_shop.ipynb" is used for illustration purposes.

## Problem description

The classic PFSP is characterized as follows:

- There is a set of $n$ jobs that need to be processed on $m$ machines.
- All jobs follow the same routing: they are processed first on machine 1, then on machine 2, and so on until machine $m$.
- All jobs must be processed in the same sequence on all machines (permutation constraint).
- Each job has a processing time on each machine.
- The objective is typically to minimize the makespan.


We can model a PFSP using PyJobShop. For each job $j$ and each machine $k$, we define a task $T_{jk}$. We need precedence constraints to ensure that task $T_{j,k-1}$ is processed before $T_{jk}$ for $k > 1$, and sequence constraints to ensure the same job ordering on all machines.

## Data

The data for a PFSP is often given by a processing times duration matrix, like this:

In [ ]:
DURATIONS = [
    [54, 79, 16, 66, 58],
    [83, 3, 89, 58, 56],
    [15, 11, 49, 31, 20],
    [71, 99, 15, 68, 85],
    [77, 56, 89, 78, 53],
    [36, 70, 45, 91, 35],
    [53, 99, 60, 13, 53],
    [38, 60, 23, 59, 41],
    [27, 5, 57, 49, 69],
    [87, 56, 64, 85, 13],
]

num_jobs, num_machines = len(DURATIONS), len(DURATIONS[0])
print(f"Problem size: {num_jobs} jobs, {num_machines} machines")

## Model

Let's now the model the PFSP. We start by adding all jobs and machines:

In [ ]:
from pyjobshop import Model

model = Model()
jobs = [model.add_job() for _ in range(num_jobs)]
machines = [model.add_machine() for _ in range(num_machines)]

For each job and machine, we create one task and its corresponding processing mode.

In [ ]:
tasks = {}  # store for later

for job_idx, job in enumerate(jobs):
    for machine_idx, machine in enumerate(machines):
        task = model.add_task(job=job)
        tasks[job_idx, machine_idx] = task

        duration = DURATIONS[job_idx][machine_idx]
        model.add_mode(task, machine, duration=duration)

We have to make sure that a job actually "flows" through the machine environment. This we can enforce by setting precedence constraints:

In [ ]:
from itertools import pairwise

for job_idx in range(num_jobs):
    for idx1, idx2 in pairwise(range(num_machines)):
        task1 = tasks[job_idx, idx1]
        task2 = tasks[job_idx, idx2]
        model.add_end_before_start(task1, task2)

Finally, we impose the same sequence constraint, which ensures that all machines process all tasks in the same order:

In [ ]:
for machine1, machine2 in pairwise(machines):
    model.add_same_sequence(machine1, machine2)

## Solving with CP-SAT callback

Let's now solve the model with OR-Tools' CP-SAT using a callback that stores all solutions found along the way.

In [ ]:
from ortools.sat.python.cp_model import CpSolverSolutionCallback

from pyjobshop.solve import solve_model_with_callback
from pyjobshop.solvers.ortools import CPModel


class SolutionCallback(CpSolverSolutionCallback):
    def __init__(self, model: CPModel):
        CpSolverSolutionCallback.__init__(self)
        self._model = model
        self.solutions = []

    def on_solution_callback(self):
        """Method that is called by ortools when a new solution is found."""
        solution = self._model.convert_to_solution(self)
        solution.objective_value = self.ObjectiveValue()
        self.solutions.append(solution)


cp_model = CPModel(model.data())
callback = SolutionCallback(cp_model)
result = solve_model_with_callback(cp_model, callback)
"""
The following also works but creates CPModel again:

result = model.solve(callback=callback)
"""
print(result)

Now let us plot the three best solutions found.

In [ ]:
from pyjobshop.plot import plot_machine_gantt

for solution in callback.solutions[-3:]:
    ax = plot_machine_gantt(solution, model.data())
    ax.set_title(f"Solution with objective value: {solution.objective_value}")

## Solving with CP Optimizer callback

Similarly, we can also solve the model with CP Optimizer using a callback that stores all solutions found along the way.


In [ ]:
from docplex.cp.solver.cpo_callback import CpoCallback

from pyjobshop.solvers.cpoptimizer import CPModel as CpoCPModel

try:
    from ortools.sat.python.cp_model import CpSolverSolutionCallback
except ImportError:
    raise ImportError("CP Optimizer is not installed.")


class CpoSolutionCallback(CpoCallback):
    def __init__(self, model: CPModel):
        self._model = model
        self.solutions = []

    def invoke(self, solver, event, sres):
        # This method is called at every solver event
        if event == "Solution":
            cpo_solution = sres.get_solution()
            solution = self._model.convert_to_solution(cpo_solution)
            solution.objective_value = cpo_solution.get_objective_values()[0]
            self.solutions.append(solution)


cpo_cp_model = CpoCPModel(model.data())
cpo_callback = CpoSolutionCallback(cpo_cp_model)
cpo_result = solve_model_with_callback(cpo_cp_model, cpo_callback)
"""
The following also works but creates CPModel again:

cpo_result = model.solve(solver="cpoptimizer", callback=cpo_callback)
"""
print(cpo_result)

for solution in cpo_callback.solutions[-3:]:
    ax = plot_machine_gantt(solution, model.data())
    ax.set_title(f"Solution with objective value: {solution.objective_value}")

## Conclusion

This notebook demonstrated how to use a callback when solving a PFSP scheduling problems using PyJobShop. 